In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("C:/Users/SINDH/Downloads/bank/bank-full.csv", sep = ";")
df.tail(15)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
45196,25,student,single,secondary,no,358,no,no,cellular,16,nov,330,1,-1,0,unknown,yes
45197,36,management,single,secondary,no,1511,yes,no,cellular,16,nov,270,1,-1,0,unknown,yes
45198,37,management,married,tertiary,no,1428,no,no,cellular,16,nov,333,2,-1,0,unknown,no
45199,34,blue-collar,single,secondary,no,1475,yes,no,cellular,16,nov,1166,3,530,12,other,no
45200,38,technician,married,secondary,no,557,yes,no,cellular,16,nov,1556,4,-1,0,unknown,yes
45201,53,management,married,tertiary,no,583,no,no,cellular,17,nov,226,1,184,4,success,yes
45202,34,admin.,single,secondary,no,557,no,no,cellular,17,nov,224,1,-1,0,unknown,yes
45203,23,student,single,tertiary,no,113,no,no,cellular,17,nov,266,1,-1,0,unknown,yes
45204,73,retired,married,secondary,no,2850,no,no,cellular,17,nov,300,1,40,8,failure,yes
45205,25,technician,single,secondary,no,505,no,yes,cellular,17,nov,386,2,-1,0,unknown,yes


In [3]:
first_column = df.pop('job')
df.insert(0, 'job', first_column)

df["Age Group"] = pd.cut(df["age"], bins=[0,30,50,70,90,float('inf')] , labels = ['Under 30', '30-50', '50-70', '70-90','90 Onwards'])

one_hot_encoded = pd.get_dummies(df["Age Group"], prefix = "Age Group")
df = pd.concat([df, one_hot_encoded], axis=1)

one_hot_encoded = pd.get_dummies(df["marital"], prefix = "Marital_status")
df = pd.concat([df, one_hot_encoded],axis=1)

one_hot_encoded = pd.get_dummies(df["education"], prefix="Education_level")
df = pd.concat([df, one_hot_encoded], axis=1)

one_hot_encoded = pd.get_dummies(df["contact"], prefix = "Contact_info")
df = pd.concat([df, one_hot_encoded], axis=1)

df["month"] = df["month"].replace({"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12})

mean_pdays = df["pdays"].mean()
df["pdays"].fillna(mean_pdays, inplace=True)

one_hot_encoded = pd.get_dummies(df["poutcome"], prefix= "Previous_outcome")
df = pd.concat([df, one_hot_encoded], axis = 1)

one_hot_encoded = pd.get_dummies(df["job"], prefix="Job_level")
df = pd.concat([df, one_hot_encoded], axis = 1)

df["y"] = df["y"].replace({"yes":1, "no":0})

df.drop(columns=['age', 'marital', 'education', 'contact','previous', 'Age Group', 'poutcome'],inplace = True)

last_column = df.pop('y')
df["y"] = last_column



df.head()

C:\Users\SINDH\AppData\Local\Temp\ipykernel_780\3601334783.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["month"] = df["month"].replace({"jan":1, "feb":2, "mar":3, "apr":4, "may":5, "jun":6, "jul":7, "aug":8, "sep":9, "oct":10, "nov":11, "dec":12})
C:\Users\SINDH\AppData\Local\Temp\ipykernel_780\3601334783.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value)

,job,default,balance,housing,loan,day,month,duration,campaign,pdays,...,Job_level_housemaid,Job_level_management,Job_level_retired,Job_level_self-employed,Job_level_services,Job_level_student,Job_level_technician,Job_level_unemployed,Job_level_unknown,y
0,management,no,2143,yes,no,5,5,261,1,-1,...,False,True,False,False,False,False,False,False,False,0
1,technician,no,29,yes,no,5,5,151,1,-1,...,False,False,False,False,False,False,True,False,False,0
2,entrepreneur,no,2,yes,yes,5,5,76,1,-1,...,False,False,False,False,False,False,False,False,False,0
3,blue-collar,no,1506,yes,no,5,5,92,1,-1,...,False,False,False,False,False,False,False,False,False,0
4,unknown,no,1,no,no,5,5,198,1,-1,...,False,False,False,False,False,False,False,False,True,0


In [4]:
df.dtypes
df["default"] = df["default"].replace({"yes":1, "no": 0})
df["housing"] = df["housing"].replace({"yes":1 , "no": 0})
df["loan"] = df["loan"].replace({"yes":1, "no":0})

C:\Users\SINDH\AppData\Local\Temp\ipykernel_780\1732103807.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["default"] = df["default"].replace({"yes":1, "no": 0})
C:\Users\SINDH\AppData\Local\Temp\ipykernel_780\1732103807.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["housing"] = df["housing"].replace({"yes":1 , "no": 0})
C:\Users\SINDH\AppData\Local\Temp\ipykernel_780\1732103807.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, 

In [5]:
df.drop(columns=['job'], inplace=True)

In [6]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
cols_to_normalize = df.columns[df.columns != 'y']
df[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])

In [28]:
my_model = Sequential([
    Dense(15, input_dim=400, activation='relu'),
    Dense(15, activation='relu'),
    Dense(1, activation='sigmoid')
])

#model.compile(optimizer='adam', loss='mean_squared_error')
my_model.compile(optimizer='Adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])
my_model.summary()
#model.save('C:/Users/SINDH/Downloads/bank/model.keras')

C:\Users\SINDH\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                     │ (None, 15)                  │           6,015 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 15)                  │             240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 1)                   │              16 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 6,271 (24.50 KB)

 Trainable params: 6,271 (24.50 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
X = df.drop(columns = ['y'])
#print(Y.shape)
Y = df["y"]
X = np.array(X)
print(X.shape)
print(X)

(45211, 40)
[[0.         0.09225936 1.         ... 0.         0.         0.        ]
 [0.         0.07306666 1.         ... 1.         0.         0.        ]
 [0.         0.07282153 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.12468905 0.         ... 0.         0.         0.        ]
 [0.         0.07886805 0.         ... 0.         0.         0.        ]
 [0.         0.09977666 0.         ... 0.         0.         0.        ]]


In [32]:
#from keras.models import load_model
#from keras.callbacks import EarlyStopping
#model = load_model("model.keras")
#
from scikeras.wrappers import KerasClassifier
estimator = KerasClassifier(model=my_model, epochs=50, batch_size=50, verbose=0)

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# now we just update our model fit call


In [33]:
estimator

KerasClassifier(
	model=<Sequential name=sequential_3, built=True>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=50
	validation_batch_size=None
	verbose=0
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=50
	class_weight=None
)

In [34]:

mse_scores = cross_val_score(estimator, X, Y, cv=kfold, scoring="neg_mean_squared_error")

mse_scores = -mse_scores

# Calculate the mean and standard deviation of MSE scores
mean_mse = np.mean(mse_scores)
std_mse = np.std(mse_scores)

# Print the results
print(f'Mean Squared Error: {mean_mse}')
print(f'Standard Deviation of Mean Squared Error: {std_mse}')

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `tf.saved_model.save()` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=C:\Users\SINDH\AppData\Local\Temp\tmpq1yhcpha.